In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain

In [2]:
import os

In [3]:
# Load models

embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    max_completion_tokens=200
)

In [4]:
# Load PDF
pdf_link = 'Lei_geral_protecao_dados_pessoais_1ed.pdf'

loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()  # Carregar o pdf e separar em páginas


In [5]:
# Separar em Chunks (Pedaços de documento)

text__splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=20,  # Número de palavras que se sobrepõem entre os chunks
    length_function=len,
    add_start_index=True
)

chunks = text__splitter.split_documents(pages)  # Separar o documento em pedaços

In [6]:
# Salvar no vector DB
db = Chroma.from_documents(chunks,embeddings_model, persist_directory="text_index")

In [8]:
# carregar DB
vectorDB = Chroma(persist_directory="text_index", embedding_function=embeddings_model)

# Load Retriever (buscador)
retriever = vectorDB.as_retriever(search_kwargs={"k": 3})  # k = número de documentos retornados

# Load QA Chain (construção da cadeia do chunks para fazer o llm)
qa_chain = load_qa_chain(
    llm,
    chain_type="stuff", # "stuff" é o tipo de cadeia que queremos usar
    verbose=True   # Mostra o que está a acontecer
)

In [11]:
def ask(question):
    context = retriever.invoke(question)  # Pega os documentos relevantes
    answer = qa_chain.invoke({"input_documents": context, "question": question})['output_text']   # Faz a pergunta
    return answer

In [12]:
user_question = input("user: ")
answer = ask(user_question) # Faz a pergunta
print(answer)  # Imprime a resposta



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Lei no 13.709/2018
Lei Geral de Proteção de Dados Pessoais (LGPD).
O PRESIDENTE DA REPÚBLICA
Faço saber que o Congresso Nacional decreta e eu sanciono a seguinte Lei:1
CAPÍTULO I – Disposições Preliminares
Art. 1 o Esta Lei dispõe sobre o tratamento de dados pessoais, inclusive 
nos meios digitais, por pessoa natural ou por pessoa jurídica de direito 
público ou privado, com o objetivo de proteger os direitos fundamentais 
de liberdade e de privacidade e o livre desenvolvimento da personalidade 
da pessoa natural.
Parágrafo único. As normas gerais contidas nesta Lei são de interesse 
nacional e devem ser observadas pela União, Estados, Distrito Federal e 
Municípios.
Art. 2o A disciplina da proteção 